# Task 5: SQL Data Migration.
Write SQL scripts to migrate the data from the lightweight database system (used for Task 3) to the SQL database you designed in Task 4

As we are using SQLite3, the fund details are stored in .db file. To perform this task, we need to convert the database into json format. 
To avoid non-value added output, we will automatically convert the .db data into json format internally using the 'read_database_to_json' 

Answer:

In [3]:
import sqlite3
import json

# Database file paths
FUNDS_DB_FILE = 'funds.db'
INVESTMENT_FUNDS_DB_FILE = 'investment_funds.db'

# Read data from an SQLite table and return it as a JSON-like Python dictionary
def read_database_to_json(db_file, table_name):
    connection = sqlite3.connect(db_file)
    cursor = connection.cursor()

    # Query all data from the specified table
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    # Fetch column names to map data properly
    column_names = [description[0] for description in cursor.description]

    # Convert to list of dictionaries
    data = [dict(zip(column_names, row)) for row in rows]
    connection.close()

    return data

# Mock Data Setup (Create tables and populate data)
# Create mock data in the funds.db and investment_funds.db databases for testing
def create_mock_data():
    # Funds Database (funds.db)
    connection = sqlite3.connect(FUNDS_DB_FILE)
    cursor = connection.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS funds (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        fund_name TEXT NOT NULL UNIQUE,
        performance REAL NOT NULL
    )
    """)
    cursor.executemany("""
    INSERT OR IGNORE INTO funds (fund_name, performance) VALUES (?, ?)
    """, [
        ("Growth Fund", 10.5),
        ("Income Fund", 7.2),
        ("Balanced Fund", 5.6)
    ])
    connection.commit()
    connection.close()

    # Investment Funds Database (investment_funds.db)
    connection = sqlite3.connect(INVESTMENT_FUNDS_DB_FILE)
    cursor = connection.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS funds (
        fund_id INTEGER PRIMARY KEY AUTOINCREMENT,
        fund_name TEXT NOT NULL UNIQUE,
        fund_type TEXT NOT NULL,
        created_date DATE NOT NULL,
        total_assets REAL NOT NULL
    )
    """)
    cursor.executemany("""
    INSERT OR IGNORE INTO funds (fund_name, fund_type, created_date, total_assets) VALUES (?, ?, ?, ?)
    """, [
        ("Growth Fund", "Equity", "2025-01-01", 5000000),
        ("Income Fund", "Bond", "2025-02-01", 3000000),
        ("Balanced Fund", "Mixed", "2025-03-01", 4000000)
    ])
    connection.commit()
    connection.close()

# Main Execution
if __name__ == "__main__":
    # Step 1: Create mock data in both databases
    create_mock_data()

    # Step 2: Read and convert data from funds.db
    funds_data = read_database_to_json(FUNDS_DB_FILE, "funds")
    print("Data from funds.db:")
    print(json.dumps(funds_data, indent=4))

    # Step 3: Read and convert data from investment_funds.db
    investment_funds_data = read_database_to_json(INVESTMENT_FUNDS_DB_FILE, "funds")
    print("\nData from investment_funds.db:")
    print(json.dumps(investment_funds_data, indent=4))


Data from funds.db:
[
    {
        "id": 1,
        "fund_name": "Testing",
        "performance": 10.0
    },
    {
        "id": 2,
        "fund_name": "Growth Fund",
        "performance": 10.5
    },
    {
        "id": 3,
        "fund_name": "Income Fund",
        "performance": 7.2
    },
    {
        "id": 4,
        "fund_name": "Balanced Fund",
        "performance": 5.6
    }
]

Data from investment_funds.db:
[
    {
        "fund_id": 2,
        "fund_name": "Growth Fund",
        "fund_type": "Equity",
        "created_date": "2025-01-01",
        "total_assets": 5000000.0
    },
    {
        "fund_id": 3,
        "fund_name": "Income Fund",
        "fund_type": "Bond",
        "created_date": "2025-02-01",
        "total_assets": 3000000.0
    },
    {
        "fund_id": 4,
        "fund_name": "Balanced Fund",
        "fund_type": "Mixed",
        "created_date": "2025-03-01",
        "total_assets": 4000000.0
    }
]
